In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

<font size=3 color='DarkBlue'> Import 2020 data where all rows with servicedate before 2018, rows with servicedate and closeddate of null are deleted

In [20]:

df311_2020=pd.read_csv(r'C:\Documents\projects\HackLA\311\data\MyLA311_Service_Request_Data_2020DT2.csv',
                       low_memory=False, index_col=0)

In [21]:
df311_2020.head(2)

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
0,1-1523593381,2020-01-01 00:02:00,2020-01-07 12:19:00,SR Created,BOS,Illegal Dumping Pickup,Closed,Call,BOS,NaN,...,"(33.988984265, -118.302681205)",673.0,J,5.0,South Los Angeles APC,8.0,Marqueece Harris-Dawson,81.0,EMPOWERMENT CONGRESS CENTRAL AREA NDC,77TH STREET
1,1-1523590121,2020-01-01 00:05:00,2020-01-02 08:43:00,SR Created,BOS,Illegal Dumping Pickup,Closed,Call,BOS,NaN,...,"(34.1940284578, -118.499471638)",531.0,D,6.0,South Valley APC,6.0,Nury Martinez,19.0,LAKE BALBOA NC,WEST VALLEY


In [22]:
df311_2020['CreatedDate'] = pd.to_datetime(df311_2020['CreatedDate'])

In [23]:
df311_2020['UpdatedDate'] = pd.to_datetime(df311_2020['UpdatedDate']) 

In [24]:
df311_2020['ServiceDate'] = pd.to_datetime(df311_2020['ServiceDate']) 

In [25]:
df311_2020['ClosedDate'] = pd.to_datetime(df311_2020['ClosedDate']) 

In [26]:
df311_2020.dtypes

SRNumber                             object
CreatedDate                  datetime64[ns]
UpdatedDate                  datetime64[ns]
ActionTaken                          object
Owner                                object
RequestType                          object
Status                               object
RequestSource                        object
CreatedByUserOrganization            object
MobileOS                             object
Anonymous                            object
AssignTo                             object
ServiceDate                  datetime64[ns]
ClosedDate                   datetime64[ns]
AddressVerified                      object
ApproximateAddress                   object
Address                              object
HouseNumber                         float64
Direction                            object
StreetName                           object
Suffix                               object
ZipCode                              object
Latitude                        

In [27]:
df311_2020.shape

(1418747, 34)

In [28]:
#Create dataframe where Servicedate before CreatedDate
dsc = df311_2020[df311_2020['ServiceDate'] < df311_2020['CreatedDate']]

In [29]:
dsc.shape

(229827, 34)

In [30]:
dsc.isnull().sum()

SRNumber                          0
CreatedDate                       0
UpdatedDate                       0
ActionTaken                       0
Owner                             0
RequestType                       0
Status                            0
RequestSource                     0
CreatedByUserOrganization         0
MobileOS                     223608
Anonymous                         0
AssignTo                         16
ServiceDate                       0
ClosedDate                        0
AddressVerified                   0
ApproximateAddress           220831
Address                           0
HouseNumber                  221241
Direction                     14198
StreetName                   221241
Suffix                        21891
ZipCode                           4
Latitude                          2
Longitude                         2
Location                          2
TBMPage                           2
TBMColumn                         2
TBMRow                      

### dsc is dataframe of rows where ServiceDate is before CreatedDate.  Below is statistics run on only rows where ServiceDate is before CreatedDate.

In [31]:
dsc['RequestType'].value_counts()

Graffiti Removal              227020
Multiple Streetlight Issue      1434
Single Streetlight Issue        1373
Name: RequestType, dtype: int64

### Most frequest request type is Graffiti Removal, then Streetlight Issue¶

In [32]:
dsc['ActionTaken'].value_counts()

SR Created                229798
Transferred                    9
SR Updated                     9
Escalate to Supervisor         9
Consultation/3-way             1
Status Provided                1
Name: ActionTaken, dtype: int64

### Most frequest ActionTaken is SR (Service Request?)¶

In [33]:
dsc['UpdatedDate'].value_counts()

2020-11-03 15:32:08    119
2020-04-04 00:05:07     43
2021-01-09 00:10:40     32
2020-12-26 00:11:30     32
2020-02-10 12:01:00     25
                      ... 
2020-04-15 19:45:57      1
2020-10-21 10:05:30      1
2020-08-12 15:10:27      1
2020-09-25 08:35:54      1
2020-04-29 11:49:42      1
Name: UpdatedDate, Length: 199928, dtype: int64

In [34]:
dsc.groupby('RequestType')['UpdatedDate'].count()

RequestType
Graffiti Removal              227020
Multiple Streetlight Issue      1434
Single Streetlight Issue        1373
Name: UpdatedDate, dtype: int64

<font size=3 color='DarkBlue'>The most frequent 'UpdatedDate' happened at exactly 2020-11-03 15:32:08, which suggests batch input.  Most of them are graffiti removal but most of all of the rows where ServiceDate is before  CreateDate is graffiti removal

In [35]:
#Owner is city agency or department assigned to the ticket
dsc['Owner'].value_counts()

OCB    227020
BSL      2807
Name: Owner, dtype: int64

### Most were assigned to the Office of Beautification, rest were assigned to the Bureau of Street Lighting (aligns with graffiti and streetlight)

In [36]:
dsc['Status'].value_counts()

Closed          228355
Cancelled         1413
Referred Out        43
Open                13
Forward              3
Name: Status, dtype: int64

#### Most were closed

In [37]:
dsc['RequestSource'].value_counts()

Driver Self Report               219665
Mobile App                         6218
Call                               2043
Self Service                       1749
Email                                94
Voicemail                            31
Council's Office                     24
Twitter                               2
Queue Initiated Customer Call         1
Name: RequestSource, dtype: int64

### Most were done by work crews for work done in field then entered into system
### Driver Self Report = By work crews for work done in field then entered into system
##### Request Source is the source through which the service request was received. Self Service = MyLA311 web portal, Web-Form = internal/external online forms

In [38]:
dsc['CreatedByUserOrganization'].value_counts()

OCB                 219921
Self Service          7876
ITA                   1899
BSL                     55
Council's Office        44
Proactive Insert        17
BSS                     15
Name: CreatedByUserOrganization, dtype: int64

### Office of Beautification created most of the requests, 2nd highest amount is through the portal or app

In [39]:
dsc['Anonymous'].value_counts()

N    226819
Y      3008
Name: Anonymous, dtype: int64

In [40]:
dsc['AssignTo'].value_counts()

CRCD        63004
GAP         30237
GAPBH       26758
CCAC        20974
HBT         19145
WVA         13368
NEGB        11946
PGS          9452
SGB          9147
NDFY         7196
NEGB_ELA     5976
LACC         5188
KYCC         4336
NORTH        1636
SOUTH         469
CENTRAL       325
WEST          289
OCB           152
GCS           125
EAST           88
Name: AssignTo, dtype: int64

	
The specific group that is assigned to this request.

In [43]:
dsc['ServiceDate'].value_counts().head(20)

2020-02-04    1473
2020-10-15    1278
2020-12-22    1124
2020-10-06    1091
2020-06-01    1085
2020-06-02     932
2020-12-15     842
2020-12-29     838
2020-12-03     837
2020-11-19     819
2020-06-04     818
2020-12-17     816
2020-05-05     807
2020-10-16     807
2020-12-28     806
2020-12-10     800
2020-12-11     794
2020-10-14     786
2020-12-02     781
2020-05-12     775
Name: ServiceDate, dtype: int64

In [45]:
#create another column that extracts the month from the 'ServiceDate'
dsc['ServiceDateMonth'] = dsc['ServiceDate'].dt.month

<ipython-input-45-b4386e5495a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsc['ServiceDateMonth'] = dsc['ServiceDate'].dt.month


In [46]:
dsc.head(2)

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct,ServiceDateMonth
160,1-1523621291,2020-01-01 09:12:00,2020-02-14 14:10:00,SR Created,BSL,Single Streetlight Issue,Closed,Self Service,Self Service,NaN,...,673.0,H,6.0,South Los Angeles APC,8.0,Marqueece Harris-Dawson,81.0,EMPOWERMENT CONGRESS CENTRAL AREA NDC,77TH STREET,1
391,1-1523650478,2020-01-01 11:27:00,2020-01-01 11:30:00,SR Created,OCB,Graffiti Removal,Closed,Driver Self Report,OCB,NaN,...,632.0,H,6.0,South Los Angeles APC,5.0,NaN,61.0,SOUTH ROBERTSON NC,WEST LOS ANGELES,12


In [47]:
dsc['ServiceDateMonth'].value_counts()

10    25490
12    24870
11    19973
1     18825
8     18771
9     17856
5     17765
7     17691
4     17583
2     17557
6     17383
3     16063
Name: ServiceDateMonth, dtype: int64

### The highest amount of calls happened at the end of the year (10-12)

In [48]:
dsc.groupby('RequestSource')['ServiceDateMonth'].count()

RequestSource
Call                               2043
Council's Office                     24
Driver Self Report               219665
Email                                94
Mobile App                         6218
Queue Initiated Customer Call         1
Self Service                       1749
Twitter                               2
Voicemail                            31
Name: ServiceDateMonth, dtype: int64

### Interesting - the info for ServiceDate on the 311 site is - The scheduled service date for SANITATION service types. Typically this will be the next scheduled trash collection date.  So is ServiceDate even relevant for graffiti and street light requests?

In [49]:
dsc['ClosedDate'].value_counts()

2020-02-10 12:01:00    25
2020-02-11 01:41:00    24
2020-01-29 17:36:00    24
2020-01-02 11:42:00    23
2020-02-19 09:56:00    23
                       ..
2020-10-12 21:15:30     1
2020-05-01 21:00:44     1
2020-12-07 11:20:30     1
2020-12-27 20:15:24     1
2020-07-10 12:30:19     1
Name: ClosedDate, Length: 198747, dtype: int64

In [51]:
#create another column that extracts the month from the 'ClosedDate'
dsc['ClosedDateMonth'] = dsc['ClosedDate'].dt.month

<ipython-input-51-331fea9e8580>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsc['ClosedDateMonth'] = dsc['ClosedDate'].dt.month


In [54]:
dsc['ClosedDateMonth'].value_counts()

10    25076
12    23399
5     21561
1     21000
6     20337
11    19216
7     18253
9     17661
8     17128
2     16348
3     15333
4     14515
Name: ClosedDateMonth, dtype: int64

In [55]:
dsc['AddressVerified'].value_counts()

Y    229825
N         2
Name: AddressVerified, dtype: int64

AddressVerified means Was the address within the City boundaries and validated with GIS data

In [56]:
dsc['ApproximateAddress'].value_counts()

N    8189
Y     807
Name: ApproximateAddress, dtype: int64

#### Most were entered with the exact address
If the location was created by using the map and moving the pin then this will be Y. If an exact address or intersection was entered this will be N.

In [77]:
dsc['Address'].value_counts()

12843 W FOOTHILL BLVD, 91342         284
5043 S NORMANDIE AVE, 90037          249
MACLAY ST AT FOOTHILL BLVD, 91342    199
3600 S MAIN ST, 90007                196
2500 S HOOPER AVE, 90011             196
                                    ... 
14360 W SHERMAN WAY, 91405             1
501 W WASHINGTON BLVD, 90015           1
10501 S SAN PEDRO ST, 90003            1
1010 N MARINE AVE, 90744               1
1523 W TORRANCE BLVD, 90501            1
Name: Address, Length: 75348, dtype: int64

In [78]:
dsc['PolicePrecinct'].value_counts()

NEWTON              48915
RAMPART             19064
77TH STREET         18191
HOLLENBECK          16243
HARBOR              14771
SOUTHEAST           14077
MISSION             13811
CENTRAL             12441
NORTHEAST           11151
HOLLYWOOD            8356
NORTH HOLLYWOOD      8309
WEST VALLEY          6515
SOUTHWEST            6074
OLYMPIC              5335
FOOTHILL             5324
PACIFIC              4611
WILSHIRE             4472
VAN NUYS             3840
TOPANGA              3106
WEST LOS ANGELES     2670
DEVONSHIRE           2545
Name: PolicePrecinct, dtype: int64